In [1]:
import numpy as np
import pandas as pd
from sodapy import Socrata
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import register_adapter, AsIs
import chart_studio.plotly as py
import plotly.graph_objects as go
from sqlalchemy import create_engine

conn = psycopg2.connect("dbname=opioids user=postgres")
cur = conn.cursor()
            
engine = create_engine('postgresql://postgres@localhost:5432/opioids')


In [2]:
client = Socrata("data.medicaid.gov", 
                 "5lSu1fL0xcZHjXcMJyWifYP5n", 
                 username = "davidby332@gmail.com", 
                 password = "Buf01r08!")

In [3]:
par_tup = (AsIs('NDC_Numeric'), AsIs('prodnme'), AsIs('gennme'))

query = sql.SQL("select %s, %s, %s from {table}").format(
                table=sql.Identifier('data_2018'))

opi_cdc = pd.read_sql_query(query, 
                           con = conn,
                           params =  par_tup)

opi_cdc['gennme'] = opi_cdc['gennme'].str.lower()

filler_words = ['hydrochloride', 'sulfate', 'phosphate', 'bitartrate']

opi_cdc['gennme'] = opi_cdc['gennme'].str.replace('/', ' ')

for word in filler_words:
    
    opi_cdc['gennme'] = opi_cdc['gennme'].str.replace(word, '')
    
opi_cdc['gennme'] = opi_cdc['gennme'].str.strip()

In [4]:
print(opi_cdc.head())

   ndc_numeric                      prodnme                  gennme
0     54017613  BUPRENORPHINE HYDROCHLORIDE           buprenorphine
1     54017713  BUPRENORPHINE HYDROCHLORIDE           buprenorphine
2     54018813       BUPRENORPHINE-NALOXONE  buprenorphine naloxone
3     54018913       BUPRENORPHINE-NALOXONE  buprenorphine naloxone
4     93360021                BUPRENORPHINE           buprenorphine


In [5]:
ndc_list = opi_cdc['ndc_numeric'].unique()

ndc_list_str = [str(ndc) for ndc in ndc_list]

ndc_df = pd.DataFrame()

state_codes = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY',
              'LA','ME','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH',
               'OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY',]

for st in state_codes:
    
    print (st)

    ndc_data = client.get("e5ds-i36p", state_code = st, limit = 1000000)

    temp_df = pd.DataFrame.from_records(ndc_data)

    temp_df = temp_df[temp_df['ndc'].isin(ndc_list_str)]
    
    temp_df = temp_df[['ndc', 'state_code', 'number_of_prescriptions', 'product_fda_list_name', 'quarter', 'units_reimbursed']]

    ndc_df = ndc_df.append(temp_df)


AL
AK
AZ
AR
CA
CO
CT
DE
FL
GA
HI
ID
IL
IN
IA
KS
KY
LA
ME
MA
MI
MN
MS
MO
MT
NE
NV
NH
NJ
NM
NY
NC
ND
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VT
VA
WA
WV
WI
WY


In [6]:
ndc_df['year'] = 2018

print(ndc_df.info())

print(ndc_df.head())

print(ndc_df['state_code'].value_counts())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84645 entries, 581 to 30366
Data columns (total 7 columns):
ndc                        84645 non-null object
state_code                 84645 non-null object
number_of_prescriptions    41191 non-null object
product_fda_list_name      84645 non-null object
quarter                    84645 non-null object
units_reimbursed           41191 non-null object
year                       84645 non-null int64
dtypes: int64(1), object(6)
memory usage: 5.2+ MB
None
              ndc state_code number_of_prescriptions product_fda_list_name  \
581   65162004710         AL                      12             OXYCODONE   
997   43386035101         AL                     NaN            Hydrocodon   
1026  68308084001         AL                     NaN             Oxycodone   
1047  59385002460         AL                     NaN            Belbuca 45   
1163  50383028793         AL                     NaN            Buprenorph   

     quarter units_reimb

In [7]:
ndc_df['ndc'] = ndc_df['ndc'].values.astype(int)

ndc_df['number_of_prescriptions'] = ndc_df['number_of_prescriptions'].values.astype(float)

opi_data = ndc_df.merge(opi_cdc, left_on = 'ndc', right_on = 'ndc_numeric')

In [10]:
opi_data.to_pickle('/Volumes/GoogleDrive/My Drive/DS/opioids/2018merged.pkl')

In [11]:
engine.execute("""DROP TABLE IF EXISTS sdud_merged;""")

opi_data.to_sql('sdud_merged', engine)

In [13]:
cur.execute(""" SELECT *
                FROM sdud_merged;
                """)

cur.fetchone()

(0,
 43386035101,
 'AL',
 None,
 'Hydrocodon',
 '3',
 None,
 2018,
 43386035101,
 'HYDROCODONE BITARTRATE-ACETAMINOPHEN',
 'acetaminophen hydrocodone')

In [14]:
cur.close()
        # commit the changes
conn.commit()